<center style><h3>Computer Assignment 1</h3></center>
<center style><h4>Informed and Uninformed Search</h4></center>
 <h4 style="text-align:right">
        محمد پویا افشاری - 810198577
</h4>

Goals:
*   Learning informed search algorithms (A*)
*   Learning uninformed search algorithms (BFS, IDS)
*   How to formulate problems (Abstraction)
*   Pros and Cons of each algorithm

In [505]:
BFS_PRINT_PATH = 0

## Importing Class:

In [506]:
import copy
from collections import Counter
from queue import Queue
import time
from typing import List
!pip install numpy
import numpy as np



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Class State:

In [507]:
class State:
    def __init__(self,edge_remain_cost) -> None:
        self.position = 0
        self.prev_state = None
        self.students_seen = []
        self.pizza_seen = []
        self.edge_remain_cost = dict.fromkeys(edge_remain_cost, 0)
        #self.loose_edges = dict.fromkeys(loose_edges, 0)
        #self.loose_remain = 0
        #self.loose_edges = loose_edges
       # self.loose_remain = 0
        #self.delivered = delivered
        self.cost = 0

    def __members(self):
        return (self.position, self.students_seen, self.pizza_seen,self.edge_remain_cost)


    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        return False

    def __hash__(self):
        return hash(f"{self.position},{self.students_seen},{self.pizza_seen},{self.cost}")

    def __str__(self) -> str:
        return f"\n cost:{self.cost}, \
                 \n position{self.position + 1} \
                 \n student:{[x+1 for x in self.students_seen]} \
                 \n pizza:{[x+1 for x in self.pizza_seen]}\
                  \n\n"  
    
    def is_order_satisfied(self, order):
        for left_student, right_student in order:
            if left_student in self.students_seen and right_student not in self.students_seen:
                continue
            if left_student not in self.students_seen and right_student in self.students_seen:
                return False
            if right_student in self.students_seen and left_student in self.students_seen and  self.students_seen.index(right_student) < self.students_seen.index(left_student):
                # if right student was seen before left student, return False
                return False
        return True


## Class Graph

In [508]:
class Graph:
    def __init__(self, v :int) -> None:
        self.v = v # number of vertices
        self.edges = [[] for _ in range(v)] #adj list
        self.student_pizza_pairs = {} # dictionary to store student-pizza pairs
        self.pizza = []
        self.students = []
        self.start_position = 0
        self.loose_edge = []
        self.order = []
        self.edgeList = []
        self.edge_remain_cost = []
        
    def add_edge(self, u :int, v :int) -> None:
        self.edges[u].append(v)
        self.edges[v].append(u)

    def is_loose_edge(self, u: int, v: int) -> bool:
        if ((u,v) in self.loose_edge or (v,u) in self.loose_edge) :
            return True

    def is_pizza(self, node :int) -> bool:
        return (node in self.pizza)

    def is_student(self, node :int) -> bool:
        return (node in self.student_pizza_pairs.keys())

    def get_neighbors(self, s :State):
        return self.edges[s.position]
    
    def add_student_pizza_pair(self, student_place: int, pizza_node: int) -> None:
        self.student_pizza_pairs[student_place] = pizza_node


## parsing Input:

In [509]:
def create_graph(path :str) -> Graph:
    file = open(path, "r")
    n, m = map(int, file.readline().split(" "))
    g = Graph(n)
    for _ in range(m):
        u, v = map(int, file.readline().split(" "))
       # edge = tuple(u,v)
        g.edgeList.append((u-1,v-1))
        g.add_edge(u -1, v -1)

    num_loose_edges = int(file.readline().strip())
    loose_edges = {}
    for _ in range(num_loose_edges):
        edge, time = map(int, file.readline().strip().split())
        if edge not in loose_edges:
            #loose_edges[edge-1]=time
            loose_edges[g.edgeList[edge-1]] = time

    g.loose_edge = loose_edges
    g.start_position = (int(file.readline())-1)
    # Create the edge_remain_cost list
    g.edge_remain_cost = {(u, v) for u, v in g.edgeList}
    
    num_students = int(file.readline().strip())
    # create a dictionary to map each student to their index in the graph's students list
    student_dict = {}
    for _ in range(num_students):
        student_place, pizza_node = map(int, file.readline().strip().split() )
        student_place = student_place -1
        pizza_node = pizza_node -1
        student_dict[_] = student_place
        g.add_student_pizza_pair(student_place, pizza_node)
        g.pizza.append(pizza_node)
        g.students.append(student_place)

    t = int(file.readline().strip())
    delivery_order = []
    for _ in range(t):
        a,b = map(int, file.readline().strip().split())
        a -= 1
        b -= 1

        if a in student_dict and b in student_dict:
            delivery_order.append((student_dict[a], student_dict[b]))
    g.order = delivery_order
    return g


In [510]:
g1 = create_graph("Test4.txt")
#g2 = create_graph("Test4.txt")
#g3 = create_graph("Test1.txt")
#g4 = create_graph("Test2.txt")
#g5 = create_graph("Test3.txt")

In [511]:
print("v: {},student_pizza_pairs:{},edges:{},student:{},pizza:{},start_position:{},order:{},loose_edge:{},edgeList:{},edge_remain_cost:{}".format(g1.v,g1.student_pizza_pairs,g1.edges,g1.students,g1.pizza,g1.start_position,g1.order,g1.loose_edge,g1.edgeList,g1.edge_remain_cost))


v: 5,student_pizza_pairs:{3: 2, 4: 1},edges:[[1], [0, 2], [1, 3, 4], [2], [2]],student:[3, 4],pizza:[2, 1],start_position:0,order:[(3, 4)],loose_edge:{(2, 3): 2, (2, 4): 3},edgeList:[(0, 1), (1, 2), (2, 3), (2, 4)],edge_remain_cost:{(0, 1), (2, 4), (1, 2), (2, 3)}


## Search Modules:


- Initial State:

create_initial_state function creates initial state which is the position which we start searching

note that there is a diffrence between states and nodes in graph that we whatch. we may be in same node but have diffrent states...

- Transition Modue:

transition(s :State, g :Graph, p :int) -> State function
gets one state and returns next-state

- Goal State:

The goal of the delivery driver is to deliver pizzas to all the students in the graph.

The reached_goal function checks if the number of students who have received their pizzas is equal to the total number of students in the graph. If this condition is true, it means that all the students have received their pizzas, and the driver has reached the goal.
    

## Initial state

In [512]:
def create_initial_state(g :Graph) -> State:
    s = State(g.edge_remain_cost)
    print(s.edge_remain_cost)
    s.position = g.start_position
    if s.position in g.pizza:
        s.pizza_seen.append(g.start_position)
    return s

initial_state1 = create_initial_state(g1)

{(0, 1): 0, (2, 4): 0, (1, 2): 0, (2, 3): 0}


## Transition module and goal state implementation:


In [513]:
counter = 0

def reached_goal(s :State, g :Graph) -> bool:
    return len(g.students) == len(s.students_seen)


def transition(s: State, g: Graph, p: int) -> State:
    global counter
    counter += 1
    next_state = copy.deepcopy(s)
    next_state.prev_state = s
    next_state.cost += 1
    


    next_state.position = p
    #check loose_edge:
    if(next_state.edge_remain_cost.get((s.position,p),0) > 0 or next_state.edge_remain_cost.get((p,s.position),0) > 0 ):
        next_state.position = s
    if(((s.position,p) in g.loose_edge or (p,s.position) in g.loose_edge )and(next_state.edge_remain_cost.get((s.position,p),0) == 0 or next_state.edge_remain_cost.get((p,s.position),0) == 0)):
        next_state.edge_remain_cost= g.loose_edge.get((s.position,p))   
    #
    if(next_state.edge_remain_cost.get((s.position,p),0) == 0 or next_state.edge_remain_cost.get((p,s.position),0) == 0):
        #if g.is_loose_edge(s,p):
        #    next_state.loose_remain = next_state.loose_remain[p]
        #    next_state.loose_edges[p] += 1
        if g.is_pizza(p) and p not in s.pizza_seen:
            if(len(s.pizza_seen) == 0):
                next_state.pizza_seen.append(p)
        if g.is_student(p) and p not in s.students_seen:
            # check if the pizza that the student wants is the pizza that it currently sees
            for student, pizza in g.student_pizza_pairs.items():
                print("s:{},p: {},pizzas:{},student:{},pizza_seen:{},students_seen:{}".format(s,p,pizza,student,s.pizza_seen,s.students_seen))
                if pizza in s.pizza_seen and student == p:
                    next_state.students_seen.append(p)
                    next_state.pizza_seen.remove(pizza)
                    break
    #decrease every remain cost > 0 by 1
    for edge, cost in next_state.edge_remain_cost.items():
        if cost > 0:
            next_state.edge_remain_cost[edge] = cost - 1

    return next_state




## Print_path function:
to print path after finding it!

In [514]:
def print_path(final_state : State) -> None:
    print("PATH::")
    s = final_state
    while True:
        print(f"{s.position + 1} <== ", end = " ")
        s = s.prev_state
        if s.prev_state == None: #s is first node
            print(f"{s.position + 1} ")
            break
         

## Test function:


In [515]:
def test(g :Graph, method) -> None:
    global counter
    counter = 0
    start = time.time()
    path = method(g, create_initial_state(g))
    end = time.time()
    print(f"time:{end - start}")
    print_path(path)
    print(f"cost:{path.cost},")
    print(f"states:{counter},")

# BFS:

BFS (Breath First Search) is an uninformed search algorithms witch will check all possible actions to find the answer.It is a simple algorithm that starts at the root node and explores all the neighboring nodes at the present depth level before moving on to the next level. It uses a queue data structure to keep track of the nodes to be explored.

This algorithm is very memory hungry and relatively slow.

We use a queue.in each turn, we dequeue from state and enqueue all succssors to queue until reaching goal state. becasu all path costs are same (1), we can get optimal soloution.

Pros:
* Completeness: BFS is complete, which means it can always find a solution if one exists
* Optimal solution: if the path cost is uniform, BFS guarantees to find the shortest path to the goal.
* Simple to understand and implement.

Cons:
* Memory-intensive: the amount of memory required can be quite large, especially for a graph with many nodes.
* Time-intensive: BFS may take a long time to find a solution, especially if the solution is deep.



In [516]:
def bfs(g :Graph, initial_state :State) -> State:
    q = []
    q.append(initial_state)
    explored = {initial_state, }
    cost = 0
    while 1:
        state = q.pop(0)
        #for test
        if state.cost > cost:
            cost = state.cost
            print(f"cost:{cost}")

        for neighbor in g.get_neighbors(state):
            t = transition(state, g, neighbor)
            if t not in explored:
                if reached_goal(t, g):
                    return t
                q.append(t)
                explored.add(t)

## BFS Test:

Test1:

In [517]:
test(g1, bfs)

{(0, 1): 0, (2, 4): 0, (1, 2): 0, (2, 3): 0}
cost:1
cost:2


TypeError: 'dict' object is not callable

# IDS:
We use dfs with height 0,1,2,3,... til we reach goal state.

This algorithms runs DFS on increasing depth limit until find the goal state. So it solves completeness problem of DFS if may DFS faces loop and never get to the goal.

Pros:

* Complete: IDS is complete, which means it can always find a solution if one exists.
* Memory-efficient: IDS uses less memory than BFS because it only keeps track of nodes within the current depth limit.
* Time-efficient: IDS is more time-efficient than BFS because it avoids exploring paths that are too deep.
(two last are true if and only if we can have good imagination of height which we might find and reach the goal)

Cons:

* Not always optimal: IDS may not always find the shortest path to the goal.
* Can be slow: IDS may still take a long time to find a solution if the search space is large.

## IDS implementation:

In [ ]:
def dfs(state :State, step :int, g :Graph):
    if reached_goal(state, g):
        return state
    if step == 0:
        return None

    for neighbor in g.get_neighbors(state):
        next_state = transition(state, g, neighbor)
        result = dfs(next_state, step - 1, g)
        if result != None:
            return result
    return None

h = 0
def ids(g:Graph, initial_state :State) -> State:
    global h
    while True:
        h += 1
        print(f"h:{h}")

        result = dfs(initial_state, h, g)
        if result != None:
            return result
        print(f"cant find in h:{h}")

## IDS Tests:

In [ ]:
h = 0
test(g1, ids)

{(0, 1): 0, (1, 2): 0, (2, 3): 0, (2, 4): 0}
h:1
cant find in h:1
h:2
cant find in h:2
h:3
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
cant find in h:3
h:4
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost

# ASTAR:

## A-Star search: 

Because number of studemts getting fed in any pass from initial state to goal state is consecutively non-decreasing. The heuristic would be consistant.

So we choose -len(s.student_seen) as our consistent huristic.

## A-Star Implementation:

A* (A-star) is a more complex algorithm that combines both BFS and heuristic functions to guide the search towards the goal. A* evaluates nodes based on both the path cost so far and an estimate of the remaining cost to the goal. The estimate is usually given by a heuristic function that approximates the distance from the current node to the goal.

Pros:

* Optimality: A* is optimal if the heuristic function is admissible, which means it never overestimates the actual cost to the goal.
* Time-efficient: A* can be much faster than BFS or IDS because it uses heuristics to guide the search.
* Memory-efficient: A* uses less memory than BFS because it only keeps track of nodes on the current path.

Cons:

* Completeness: A* is not guaranteed to be complete, especially if the heuristic function is not admissible.
* Heuristic function: A* requires a good heuristic function to be effective, and designing a good heuristic function can be challenging.
* Implementation complexity: A* is more complex to implement than BFS or IDS.

we have two lists, explored and frountier.

we choose state s with minimum f(s) = h(s) + cost(s) from frontier and explore its successors.
    next_state = min(frountier, key = lambda s : huristic(s) + s.cost) 

and we remember explored states to make sure not to rewatch them.

it will give us optimal soloution if we make sure our huristic is consistent.

In [ ]:
alpha = 1

def huristic(s :State) -> int:
    global alpha
    return -len(s.students_seen) * alpha

def a_star(g:Graph, initial_state :State) -> State:
    explored = []
    frountier = [initial_state, ]
    cost = 0

    while True:
        next_state = min(frountier, key = lambda s : huristic(s) + s.cost)
        frountier.remove(next_state)
        explored.append(next_state)

        if next_state.cost > cost:
            cost = next_state.cost
            print(f"cost:{cost}")

        if reached_goal(next_state, g):
            return next_state
        for neighbor in g.get_neighbors(next_state):
            t = transition(next_state, g, neighbor)
            if t not in explored and t not in frountier:
                frountier.append(t)

## Test A-Star

In [ ]:
alpha = 1

### Test1

In [ ]:
test(g1 , a_star)

{(0, 1): 0, (1, 2): 0, (2, 3): 0, (2, 4): 0}
cost:1
cost:2
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
cost:3
cost:4
s:
 cost:4,                  
 position3                  
 student:[5]                  
 pizza:[3]                  

,p: 3,pizzas:2,student:3,pizza_seen:[2],students_seen:[4]
cost:5
time:0.0020041465759277344
PATH::
4 <=

## Test Weighted A-Star1:
* alpha = 5

In [ ]:
alpha = 5

### Test1

In [ ]:
test(g1 , a_star)

{(0, 1): 0, (1, 2): 0, (2, 3): 0, (2, 4): 0}
cost:1
cost:2
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
cost:3
cost:4
s:
 cost:4,                  
 position3                  
 student:[5]                  
 pizza:[3]                  

,p: 3,pizzas:2,student:3,pizza_seen:[2],students_seen:[4]
cost:5
time:0.0010018348693847656
PATH::
4 <=

## Test Weighted A-Star2:
* alpha = 10

In [ ]:
alpha = 10

### Test1

In [ ]:
test(g1 , a_star)

{(0, 1): 0, (1, 2): 0, (2, 3): 0, (2, 4): 0}
cost:1
cost:2
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 3,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:2,student:3,pizza_seen:[1],students_seen:[]
s:
 cost:2,                  
 position3                  
 student:[]                  
 pizza:[2]                  

,p: 4,pizzas:1,student:4,pizza_seen:[1],students_seen:[]
cost:3
cost:4
s:
 cost:4,                  
 position3                  
 student:[5]                  
 pizza:[3]                  

,p: 3,pizzas:2,student:3,pizza_seen:[2],students_seen:[4]
cost:5
time:0.002000093460083008
PATH::
4 <==